In [1]:
from utils.notebook_2_utils import * 
import utils.notebook_2_utils as utils

training_df = pd.read_csv("data/training.csv")
lookup_df = pd.read_csv("data/IdLookupTable.csv")
training_df.fillna(method = 'ffill',inplace = True)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
# training_df

In [3]:
train_X, train_Y, val_X, val_Y = create_train_test_sets_nchw(training_df, normalize=True)

In [4]:
train_X.shape, train_Y.shape, val_X.shape, val_Y.shape, train_X.max(), train_Y.max(), val_Y.max()

(torch.Size([5640, 1, 96, 96]),
 torch.Size([5640, 30]),
 torch.Size([1409, 1, 96, 96]),
 torch.Size([1409, 30]),
 tensor(1.),
 tensor(0.9993),
 tensor(0.9941))

## Very Simple CNN Model
* Input shape:  64, 1, 96, 96 (nchw)
* Output shape: 64, 30 (nc)


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
# from sklearn.model_selection import train_test_split
import torch.nn.functional as F

### Model Definition

In [6]:
class CNN(nn.Module):
    def __init__(self, input_shape: int, hidden_dim: int, output_dim: int):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=input_shape, out_channels=hidden_dim, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=hidden_dim, out_channels=hidden_dim, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(hidden_dim * 24 * 24, hidden_dim) #this is multiplied by 7*7 because the image is 28*28 and we have 2 conv layers with stride 1 and padding 1
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Control Panel

In [7]:
cnn = CNN(input_shape=1, hidden_dim=64, output_dim=30)

EPOCHS = 60
LEARNING_RATE = .02
BATCH_SIZE = 32


loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(cnn.parameters(), lr=LEARNING_RATE)





### DataLoaders

In [8]:
torch.manual_seed(42)
from torch.utils.data import TensorDataset, DataLoader


# Set number of epochs


# Put data to target device
train_X, train_Y = train_X.to(device), train_Y.to(device)
val_X, val_Y = val_X.to(device), val_Y.to(device)
cnn = cnn.to(device)

train_dataset = TensorDataset(train_X, train_Y)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = TensorDataset(val_X, val_Y)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [9]:
for X_batch, y_batch in train_dataloader:
    print(X_batch.shape, y_batch.shape)
    break

torch.Size([32, 1, 96, 96]) torch.Size([32, 30])


In [10]:
for X_batch, y_batch in test_dataloader:
    print(X_batch.shape, y_batch.shape)
    break

torch.Size([32, 1, 96, 96]) torch.Size([32, 30])


In [11]:
for idx,batch in enumerate(train_dataloader):
    print("batch input: ", batch[0].size())
    print("batch idx: ",  idx)
    print("batch label: " , batch[1].shape)
    break

print("---------- test dataloader -------")
for idx, (data,target) in enumerate(test_dataloader):
    print(idx, data.shape)
    print(idx, target.shape)
    break

batch input:  torch.Size([32, 1, 96, 96])
batch idx:  0
batch label:  torch.Size([32, 30])
---------- test dataloader -------
0 torch.Size([32, 1, 96, 96])
0 torch.Size([32, 30])


In [12]:
preds = cnn(X_batch)

In [13]:
y_batch.shape

torch.Size([32, 30])

In [14]:
loss_fn(preds, y_batch)

tensor(0.2982, device='cuda:0', grad_fn=<MseLossBackward0>)

## Training Loop (No gradient accumilation, see below for loop that's used)

In [15]:
for epoch in range(EPOCHS):
    ### Training mode 
    cnn.train()

    
    for X_batch, y_batch in train_dataloader:
        # 1. Forward pass (model outputs raw logits)
        y_logits = cnn(X_batch)
        
        # 2. Calculate loss/accuracy
        loss = loss_fn(y_logits, y_batch)
        
        # 3. Optimizer zero grad
        optimizer.zero_grad()
        
        # 4. Loss backwards
        loss.backward()
        
        # 5. Optimizer step
        optimizer.step()
    
    
    cnn.eval()
    with torch.inference_mode():
        for X_batch, y_batch in test_dataloader:
            test_logits = cnn(X_batch)
            test_loss = loss_fn(test_logits, y_batch)

    # Print out what's happening
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f} | Test Loss: {test_loss:.5f}")

Epoch: 0 | Loss: 0.00489 | Test Loss: 0.00172
Epoch: 10 | Loss: 0.00196 | Test Loss: 0.00089
Epoch: 20 | Loss: 0.00095 | Test Loss: 0.00078
Epoch: 30 | Loss: 0.00376 | Test Loss: 0.00095
Epoch: 40 | Loss: 0.00069 | Test Loss: 0.00025
Epoch: 50 | Loss: 0.00053 | Test Loss: 0.00030


## Viewing a prediction

In [20]:
torch.cuda.empty_cache()

## **THIS** error is why we use gradient accumilation, see next notebook (2.5)

In [21]:
preds = cnn(val_X.to(device))

RuntimeError: CUDA out of memory. Tried to allocate 3.10 GiB (GPU 0; 15.74 GiB total capacity; 3.51 GiB already allocated; 1.17 GiB free; 3.61 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
loss_fn(preds.to(device), val_Y.to(device))

In [ ]:
val_X.shape, preds.shape, val_Y.shape

In [ ]:
preds[0][1]

In [ ]:
val_X[1].shape, val_X[1][0].shape

In [ ]:
def show_pred(X, preds, actual, index, point):
    plt.imshow(X[index][0],cmap='gray')

    plt.scatter(96* preds[index][point],96* preds[index][point + 1] ,c='r', marker='s', s=60, alpha=.5)
    plt.scatter(96*actual[index][point],96* actual[index][point+1],c='g', marker='s', s=60, alpha=.5)

    plt.legend(['predicted','actual'])
    


In [ ]:
show_pred(val_X.to("cpu"), preds.to("cpu").detach().numpy(), val_Y.to("cpu").detach().numpy(), 0, 0)

In [ ]:
preds[0][0], preds[0][1]

In [ ]:
val_Y[0][0], val_Y[0][1]

In [ ]:
val_X[0][0]

## RESNET